***Load Charmader Utilities***

In [13]:
import org.att.charmander.CharmanderUtils

import org.att.charmander.CharmanderUtils

&nbsp;

***See what tasks are currently running and pick first test***

In [14]:
val runningSimulators = CharmanderUtils.getMeteredTaskNamesFromRedis
val simulatorToInvestigate = runningSimulators(0)

runningSimulators: List[String] = List(stress60)
simulatorToInvestigate: String = stress60

&nbsp;

***Retrieve memory-usage datapoints for the first simulator***

In [15]:
val memoryUsage= CharmanderUtils.getRDDForTask(sc, simulatorToInvestigate, "memory_usage", 100)

memoryUsage: org.apache.spark.rdd.RDD[List[BigDecimal]] = stress60 ParallelCollectionRDD[23] at parallelize at CharmanderUtils.scala:141

&nbsp;

***Just for curiosity lets look at the first data point***

In [16]:
memoryUsage.first

res19: List[BigDecimal] = List(1422585362000, 540000001, 33447936)

&nbsp;

***Translate Datapoints in to a Spark-SQL Database***

In [17]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext._

case class MemoryUsage(timestamp: BigDecimal, memory: BigDecimal)

val memoryusageRdd = memoryUsage.map(p => MemoryUsage(BigDecimal(p(0).asInstanceOf[BigInt]), BigDecimal(p(2).asInstanceOf[BigInt])))
memoryusageRdd.registerTempTable("memoryusage")


sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@40be3b0d
import sqlContext._
defined class MemoryUsage
memoryusageRdd: org.apache.spark.rdd.RDD[MemoryUsage] = MappedRDD[24] at map at <console>:83

&nbsp;

***Get max-memory usage***

In [18]:
val newestMax = sqlContext.sql("select max(memory) from memoryusage").first()

newestMax: org.apache.spark.sql.Row = [63070208]

&nbsp;

***Set the new max value in our task intelligence database***

In [21]:
CharmanderUtils.setTaskIntelligence(simulatorToInvestigate, "mem", newestMax(0).toString)

***To Verify that the value is set***

Visit [redis](http://172.31.2.11:31610/) and look for the task-intelligence section
